## MNIST Digits Classification with Machine Learning:
The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes.

Our goal would be to build a neural network with 2 hidden layers.

### Import the necessary Python libraries:

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfdt  
#tensorflow's data provider for MNIST

### Load the dataset in the 2-tuple structure:

In [2]:
mnist_dataset, mnist_info = tfdt.load(name='MNIST',with_info=True,as_supervised = True)

### Extract the training and testing dataset and create the validation dataset:

In [3]:
# once we have loaded the dataset, we can easily extract the training and testing dataset with the built references
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']


num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# .num_examples -> It contains the number of images in the test data set.
# mnist_info.splits['test'].num_examples -> returns 10,000, which is the number of samples (images) in the test split of the mnist data set.

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

## Preprocessing

### Scaling the image:

In [4]:
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

### Shuffling and batching the dataset

In [5]:
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outlining the model:

In [6]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

### Optimizing the model:

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


### Training the model:

In [8]:
NUM_EPOCHS = 5  # do it 6 and see different

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 3s - loss: 0.3452 - accuracy: 0.9007 - val_loss: 0.1767 - val_accuracy: 0.9482 - 3s/epoch - 5ms/step
Epoch 2/5
540/540 - 2s - loss: 0.1398 - accuracy: 0.9585 - val_loss: 0.1224 - val_accuracy: 0.9647 - 2s/epoch - 3ms/step
Epoch 3/5
540/540 - 2s - loss: 0.1017 - accuracy: 0.9690 - val_loss: 0.1134 - val_accuracy: 0.9677 - 2s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0794 - accuracy: 0.9755 - val_loss: 0.0932 - val_accuracy: 0.9717 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 1s - loss: 0.0633 - accuracy: 0.9804 - val_loss: 0.0737 - val_accuracy: 0.9762 - 1s/epoch - 3ms/step


### Evaluating the result:

In [9]:
test_loss, test_accuracy = model.evaluate(test_data)


1/1 [==============================] - 0s 259ms/step - loss: 0.0847 - accuracy: 0.9749


In [10]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.49%
